## Import Libaries

In [3]:
# import all necessary libraries
import os
import datetime
from utils.Copernicus import AdvancedCopernicus
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import folium
from tqdm import tqdm
from pathlib import Path
from utils.OpenMeteoWeather import OpenMeteoWeatherHistory  
import math

from datetime import datetime
# Ignore SettingWithCopyWarning:
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)



# Display all columns
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [ ]:
START_DATE = '2022-12-03'
END_DATE = '2025-03-22'
OUTPUT_FILENAME = 'output.nc'

BBOX= {
    
    # "minimum_longitude":9.10,
    # "maximum_longitude":30.20,
    # "minimum_latitude":53.10,
    # "maximum_latitude":65.00,

    "minimum_longitude":9.1,
    "maximum_longitude":23.0,
    "minimum_latitude":53.5,
    "maximum_latitude":60.0,

    # "minimum_longitude":9.85083510071235,
    # "maximum_longitude":10.926709174713364,
    # "minimum_latitude":54.25206332481298,
    # "maximum_latitude":54.97306793985031,

    "target_min_lon":10.156,
    "target_max_lon":10.170,
    "target_min_lat":54.354,
    "target_max_lat":54.365
    }

In [5]:
# Download Data
minimum_latitude = BBOX['minimum_latitude']
maximum_latitude = BBOX['maximum_latitude']
minimum_longitude = BBOX['minimum_longitude']
maximum_longitude = BBOX['maximum_longitude']

copernicus = AdvancedCopernicus()
points = 10
latitudes = np.linspace(minimum_latitude, maximum_latitude, points)
longitudes = np.linspace(minimum_longitude, maximum_longitude, points)

In [7]:
# zip latitudes and longitudes
coordinates = set([(float(lat), float(lon)) for lat in latitudes for lon in longitudes])
print(len(coordinates))
# plot the coordinates to folium
folium_map = folium.Map(location=[minimum_latitude, minimum_longitude], zoom_start=4)
for lat, lon in coordinates:
    folium.CircleMarker(location=[lat, lon], radius=4, color='red').add_to(folium_map)
folium_map

100


### Ocean Data

In [5]:
numerical_data_path = Path(f'../data/numerical_data/points{points}')


In [6]:
# Daten die mit .nc enden
nc_files = [file for file in os.listdir(numerical_data_path) if file.endswith('.nc')]
nc_files = sorted(nc_files)
nc_files

['cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat54.092-lon14.375_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat55.083-lon10.858_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat55.083-lon12.617_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat55.083-lon14.375_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat55.083-lon16.133_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat55.083-lon17.892_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat55.083-lon19.65_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat56.075-lon10.858_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat56.075-lon12.617_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat56.075-lon16.133_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2022-12-03-2025-03-22_lat56.075-lon17.892_points13.nc',
 'cmems_mod_bal_phy_anfc_PT1H-i_2

In [7]:
df_ocean = pd.DataFrame()
for file in nc_files:
    data = xr.open_dataset(numerical_data_path / file)
    df = data.to_dataframe().reset_index()
    df_ocean = pd.concat([df_ocean, df], axis=0)
    data.close()
    # break


df_ocean

,depth,latitude,longitude,time,bottomT,mlotst,siconc,sithick,sla,so,sob,thetao,uo,vo,wo
0,0.501646,54.091614,14.374895,2022-12-03 00:00:00,7.341587,10.008370,0.000000,0.000000,-0.139298,7.649167,8.102526,6.708968,-0.037489,0.074989,1.408814e-06
1,0.501646,54.091614,14.374895,2022-12-03 01:00:00,7.349719,10.008045,0.000000,0.000000,-0.139712,7.649509,8.103182,6.709668,-0.029160,0.076673,1.986998e-06
2,0.501646,54.091614,14.374895,2022-12-03 02:00:00,7.343012,10.000727,0.000000,0.000000,-0.149037,7.649466,8.104274,6.690140,-0.018730,0.080867,2.061850e-06
3,0.501646,54.091614,14.374895,2022-12-03 03:00:00,7.334803,9.998863,0.000000,0.000000,-0.151411,7.652840,8.105758,6.673232,-0.006722,0.076700,2.182848e-06
4,0.501646,54.091614,14.374895,2022-12-03 04:00:00,7.326393,10.008844,0.000000,0.000000,-0.138694,7.659944,8.107533,6.660472,0.005799,0.062841,2.223396e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20156,0.501646,65.007874,24.930717,2025-03-21 20:00:00,-0.113599,4.928304,0.964335,0.511807,-0.038616,2.262715,2.804044,2.511556,-0.000592,-0.001702,-8.082750e-08
20157,0.501646,65.007874,24.930717,2025-03-21 21:00:00,-0.113652,4.929451,0.962926,0.512123,-0.037469,2.261493,2.803944,2.430133,-0.003716,-0.000551,-1.029538e-07
20158,0.501646,65.007874,24.930717,2025-03-21 22:00:00,-0.113722,4.930955,0.961550,0.512519,-0.035965,2.261378,2.803786,2.347729,-0.001797,0.000249,-7.995400e-08
20159,0.501646,65.007874,24.930717,2025-03-21 23:00:00,-0.113702,4.939600,0.960216,0.512811,-0.027320,2.262864,2.803596,2.249189,0.000052,0.003480,-9.934801e-08


In [8]:
def process_df(df:pd.DataFrame)->pd.DataFrame:
    # df = df.dropna(axis=1, how="all")
    # df = df.dropna(axis=0, how="any")
    # drop cols with lot of missing values
    #df = df.drop(columns=['depth'], axis=1)
    df = df[["time"] + [col for col in df.columns if col != "time"]]
    float_cols = df.select_dtypes(include=["float"]).columns
    df[float_cols] = df[float_cols].astype(np.float32)
    df["time"] = pd.to_datetime(df["time"]).dt.tz_localize(None).dt.round("h")
    
    #df = df.groupby(by=['time', 'latitude', 'longitude']).mean().reset_index()
    df = df.reset_index(drop=True)

    return df

In [9]:
df_ocean.columns

Index(['depth', 'latitude', 'longitude', 'time', 'bottomT', 'mlotst', 'siconc',
       'sithick', 'sla', 'so', 'sob', 'thetao', 'uo', 'vo', 'wo'],
      dtype='object')

In [10]:
df_ocean = df_ocean.dropna(axis=1, how="all")
df_ocean = df_ocean.dropna(axis=0, how="any")
df_ocean = df_ocean.drop(columns=['depth'], axis=1)
df_ocean = process_df(df_ocean)
df_ocean.info()
df_ocean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169338 entries, 0 to 1169337
Data columns (total 14 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   time       1169338 non-null  datetime64[ns]
 1   latitude   1169338 non-null  float32       
 2   longitude  1169338 non-null  float32       
 3   bottomT    1169338 non-null  float32       
 4   mlotst     1169338 non-null  float32       
 5   siconc     1169338 non-null  float32       
 6   sithick    1169338 non-null  float32       
 7   sla        1169338 non-null  float32       
 8   so         1169338 non-null  float32       
 9   sob        1169338 non-null  float32       
 10  thetao     1169338 non-null  float32       
 11  uo         1169338 non-null  float32       
 12  vo         1169338 non-null  float32       
 13  wo         1169338 non-null  float32       
dtypes: datetime64[ns](1), float32(13)
memory usage: 66.9 MB


,time,latitude,longitude,bottomT,mlotst,siconc,sithick,sla,so,sob,thetao,uo,vo,wo
0,2022-12-03 00:00:00,54.091614,14.374895,7.341587,10.008370,0.0,0.0,-0.139298,7.649167,8.102526,6.708968,-0.037489,0.074989,0.000001
1,2022-12-03 01:00:00,54.091614,14.374895,7.349719,10.008045,0.0,0.0,-0.139712,7.649509,8.103182,6.709668,-0.029160,0.076673,0.000002
2,2022-12-03 02:00:00,54.091614,14.374895,7.343012,10.000727,0.0,0.0,-0.149037,7.649466,8.104274,6.690140,-0.018730,0.080867,0.000002
3,2022-12-03 03:00:00,54.091614,14.374895,7.334803,9.998863,0.0,0.0,-0.151411,7.652840,8.105758,6.673232,-0.006722,0.076700,0.000002
4,2022-12-03 04:00:00,54.091614,14.374895,7.326393,10.008844,0.0,0.0,-0.138694,7.659944,8.107533,6.660472,0.005799,0.062841,0.000002


In [11]:
file_path = Path(numerical_data_path, 'df_ocean.csv')
# print(f"Saving data to {file_path}")
# df_ocean.to_csv(file_path, index=False)

In [12]:
import folium

df_map = df_ocean.groupby(by=["latitude", "longitude"]).mean().reset_index()
# Create a folium map centered around the average lat/lon
map_center = [df_map["latitude"].mean(), df_map["longitude"].mean()]
m = folium.Map(location=map_center, zoom_start=5)



df_map = df_ocean.groupby(by=["latitude", "longitude"]).mean().reset_index()

for _, row in df_map.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        #popup=f"Time: {row['time']}<br>Depth: {row['depth']}<br>BottomT: {row['bottomT']}",
        tooltip=f"Lat: {row['latitude']}, Lon: {row['longitude']}",
        icon=folium.Icon(color="red")
    ).add_to(m)


m

### Weather Data

In [13]:
# Get unique latitudes and longitudes
coordinates = sorted(set(zip(df_ocean['latitude'], df_ocean['longitude'])))
print(f'Number of unique coordinates: {len(coordinates)}')

list(coordinates)

Number of unique coordinates: 58


[(54.09161376953125, 14.374895095825195),
 (55.07493209838867, 10.84713077545166),
 (55.07493209838867, 12.62490177154541),
 (55.07493209838867, 14.374895095825195),
 (55.07493209838867, 16.124893188476562),
 (55.07493209838867, 17.87494659423828),
 (55.07493209838867, 19.65277862548828),
 (55.09159851074219, 10.84713077545166),
 (55.09159851074219, 12.62490177154541),
 (55.09159851074219, 14.374895095825195),
 (55.09159851074219, 16.124893188476562),
 (55.09159851074219, 17.87494659423828),
 (55.09159851074219, 19.65277862548828),
 (56.07491683959961, 10.84713077545166),
 (56.07491683959961, 12.62490177154541),
 (56.07491683959961, 16.124893188476562),
 (56.07491683959961, 17.87494659423828),
 (56.07491683959961, 19.65277862548828),
 (57.05823516845703, 10.84713077545166),
 (57.05823516845703, 17.87494659423828),
 (57.05823516845703, 19.65277862548828),
 (57.05823516845703, 21.40283203125),
 (57.07490158081055, 10.84713077545166),
 (57.07490158081055, 17.87494659423828),
 (57.07490158

In [ ]:
latitudes = [lat for lat, lon in coordinates]
longitudes = [lon for lat, lon in coordinates]

    
print(latitudes, longitudes)
end_date_dt = datetime.strptime(END_DATE, "%Y-%m-%d")  # angepasstes Format
start_date_dt = datetime.strptime(START_DATE, "%Y-%m-%d")  # angepasstes Format

# Optional: Wenn Sie Zeit hinzufügen möchten, können Sie dies manuell tun
# start_date_dt = start_date_dt.replace(hour=0, minute=0, second=0)
# end_date_dt = end_date_dt.replace(hour=23, minute=59, second=59)

end_str = end_date_dt.strftime("%Y-%m-%d")
start_str = start_date_dt.strftime("%Y-%m-%d")

open_meteo_weather = OpenMeteoWeather(
    latitudes=latitudes,
    longitudes=longitudes,
    start_date=start_str,
    end_date=end_str
)
df_weather = open_meteo_weather.get_weather_dataframe()

[54.09161376953125, 55.07493209838867, 55.07493209838867, 55.07493209838867, 55.07493209838867, 55.07493209838867, 55.07493209838867, 55.09159851074219, 55.09159851074219, 55.09159851074219, 55.09159851074219, 55.09159851074219, 55.09159851074219, 56.07491683959961, 56.07491683959961, 56.07491683959961, 56.07491683959961, 56.07491683959961, 57.05823516845703, 57.05823516845703, 57.05823516845703, 57.05823516845703, 57.07490158081055, 57.07490158081055, 57.07490158081055, 57.07490158081055, 58.05821228027344, 58.05821228027344, 58.05821228027344, 58.05821228027344, 58.05821228027344, 58.05821228027344, 59.04153060913086, 59.04153060913086, 59.04153060913086, 59.04153060913086, 59.058197021484375, 59.058197021484375, 59.058197021484375, 59.058197021484375, 60.0415153503418, 60.0415153503418, 60.0415153503418, 60.0415153503418, 60.0415153503418, 61.02483367919922, 61.02483367919922, 61.041500091552734, 61.041500091552734, 62.024818420410156, 62.024818420410156, 63.00813674926758, 63.02480

In [ ]:
df_weather.info()
df_weather.head()

In [ ]:
drop_cols = ['snow_depth']
df_weather.drop(columns=drop_cols, inplace=True)

In [ ]:
df_weather_cleaned = df_weather.dropna(axis=1, how="all")
df_weather_cleaned = df_weather_cleaned.dropna(axis=0, how="any")
df_weather_cleaned = process_df(df_weather_cleaned)
df_weather_cleaned

In [ ]:
cords = set(zip(df_weather_cleaned['latitude'].astype(float), df_weather_cleaned['longitude'].astype(float)))
cords

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Figure mit Kartenprojektion erstellen
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': ccrs.PlateCarree()})

# Küstenlinien und Länder hinzufügen
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Punkte plotten
ax.scatter(df_weather_cleaned['longitude'], df_weather_cleaned['latitude'], c='red', s=10, alpha=0.7, transform=ccrs.PlateCarree())

# Achsen und Titel
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Kartendarstellung der Koordinaten")

plt.show()

In [ ]:
df_weather_cleaned['latitude'].nunique(), df_weather_cleaned['longitude'].nunique()
#df_weather['time'].nunique()

In [ ]:



#df_weather_cleaned.rename(columns={'latitude': 'latitude_weather', 'longitude': 'longitude_weather'}, inplace=True)
display(df_weather_cleaned.head())
df_weather_cleaned.info()

## Plot Ocean and Weather Data Coordinates and Target

In [ ]:
import folium

df_map = df_weather_cleaned.groupby(by=["latitude", "longitude"]).mean().reset_index()
# Create a folium map centered around the average lat/lon
map_center = [df_map["latitude"].mean(), df_map["longitude"].mean()]
m = folium.Map(location=map_center, zoom_start=5)

# Add markers
for _, row in df_map.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        #popup=f"Time: {row['time']}<br>Depth: {row['depth']}<br>BottomT: {row['bottomT']}",
        tooltip=f"Lat: {row['latitude']}, Lon: {row['longitude']}",
        icon=folium.Icon(color="blue")
    ).add_to(m)

df_map = df_ocean.groupby(by=["latitude", "longitude"]).mean().reset_index()

for _, row in df_map.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        #popup=f"Time: {row['time']}<br>Depth: {row['depth']}<br>BottomT: {row['bottomT']}",
        tooltip=f"Lat: {row['latitude']}, Lon: {row['longitude']}",
        icon=folium.Icon(color="red")
    ).add_to(m)



m

## Merge Ocean Data and Weater Data

In [ ]:
display(df_ocean.head())
display(df_weather_cleaned.head())

In [ ]:
df_ocean_weather = pd.merge(df_ocean, df_weather_cleaned, on=['time', 'latitude', 'longitude'], how='inner')
#df_ocean_weather.groupby(by=['time', 'latitude', 'longitude']).mean().reset_index()
df_ocean_weather.info()
df_ocean_weather.head()

## Merge Ocean/Weather Data with Target Data

## Save Data for Processing

In [ ]:
# Save Data as json and csv
file_path = Path(numerical_data_path, f'df_ocean_weather_points{points}.json')
df_ocean_weather.to_json(file_path, orient='records', lines=True, date_format='iso', date_unit='s')

file_path = Path(numerical_data_path, f'df_ocean_weather_points{points}.csv')
df_ocean_weather.to_csv(file_path, index=False, sep='\t')